In [1]:
import os, json
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

BASE    = "/ShangGaoAIProjects/Lingge/LINCS/data"
OUT_DIR = f"{BASE}/Processed_data/L1000gctx_process"

CAT_PATH = f"{OUT_DIR}/Y_landmark_train_catalog_canonical.index.parquet"  # 与 Y 行对齐的 catalog
CAT = pd.read_parquet(CAT_PATH)

SMI_COL = "smiles_canonical" if "smiles_canonical" in CAT.columns else "smiles"
assert SMI_COL in CAT.columns, f"catalog 缺少 {SMI_COL} 列"
print("Catalog shape:", CAT.shape, "| using:", SMI_COL)


Catalog shape: (209540, 18) | using: smiles_canonical


In [2]:
CAT

,sig_id,phase,pert_id,pert_iname,cell_id,time_h,dose_uM,dose_value,dose_unit_raw,smiles,smiles_canonical,inchi_key,compound_id,is_small_molecule,is_control,pert_type,col_pos,dataset
0,AML001_CD34_24H:BRD-A03772856:0.37037,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,0.37037,0.37037,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434316,GSE92742
1,AML001_CD34_24H:BRD-A03772856:1.11111,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,1.11111,1.11111,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434315,GSE92742
2,AML001_CD34_24H:BRD-A03772856:10,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,10.00000,10.00000,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434313,GSE92742
3,AML001_CD34_24H:BRD-A03772856:3.33333,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,3.33333,3.33333,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434314,GSE92742
4,AML001_CD34_24H:BRD-A19037878:1.11111,GSE92742,BRD-A19037878,trichostatin-a,cd34,24.0,1.11111,1.11111,µM,CC(\C=C(C)\C=C\C(=O)NO)C(=O)c1ccc(cc1)N(C)C,CC(C=CC(=O)NO)=CC(C)C(=O)c1ccc(N(C)C)cc1,RTKIYFITIVXBLE-WKWSCTOISA-N,RTKIYFITIVXBLE-WKWSCTOISA-N,True,False,trt_cp,434463,GSE92742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209535,REP.A028_YAPC_24H:K09,GSE70138,BRD-K60230970,MG-132,yapc,24.0,20.00000,20.00000,um,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,TZYWCYJVHRLUCT-VABKMULXSA-N,TZYWCYJVHRLUCT-VABKMULXSA-N,True,False,trt_cp,73903,GSE70138
209536,REP.A028_YAPC_24H:M18,GSE70138,BRD-K96862998,pirfenidone,yapc,24.0,0.04000,0.04000,um,Cc1ccc(=O)n(c1)-c1ccccc1,Cc1ccc(=O)n(-c2ccccc2)c1,ISWRGOKTTBVCFA-UHFFFAOYSA-N,ISWRGOKTTBVCFA-UHFFFAOYSA-N,True,False,trt_cp,73904,GSE70138
209537,REP.A028_YAPC_24H:O01,GSE70138,BRD-K60230970,MG-132,yapc,24.0,20.00000,20.00000,um,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,TZYWCYJVHRLUCT-VABKMULXSA-N,TZYWCYJVHRLUCT-VABKMULXSA-N,True,False,trt_cp,73905,GSE70138
209538,REP.A028_YAPC_24H:O06,GSE70138,BRD-K60230970,MG-132,yapc,24.0,20.00000,20.00000,um,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,TZYWCYJVHRLUCT-VABKMULXSA-N,TZYWCYJVHRLUCT-VABKMULXSA-N,True,False,trt_cp,73906,GSE70138


In [3]:
# 仅用 canonical SMILES 直接生成 scaffold；不做去盐、互变体处理等——你之前已经做过
def smiles_to_scaffold(s):
    if not isinstance(s, str) or not s or s == "-666":
        return ""
    m = Chem.MolFromSmiles(s)
    if m is None:
        return ""
    scaf = MurckoScaffold.GetScaffoldForMol(m)
    return Chem.MolToSmiles(scaf, isomericSmiles=False) if scaf is not None else ""

CAT = CAT.copy()
CAT["scaffold"] = CAT[SMI_COL].map(smiles_to_scaffold)
print("n_unique scaffolds:", CAT["scaffold"].nunique())
print(CAT["scaffold"].value_counts().head(5))


n_unique scaffolds: 5980
scaffold
c1ccccc1                       10580
                                2895
O=C(NCCc1ccccc1)C1=CNC=CN1      2315
c1ccc2c(c1)CCC1C2CCC2CCCC21      973
c1ccc(Nc2ccccc2)cc1              917
Name: count, dtype: int64


In [6]:
CAT.to_parquet(f"{OUT_DIR}/Y_landmark_train_catalog_canonical_with_scaffolds.index.parquet", index=False)

In [8]:
import os, json
import numpy as np
import pandas as pd

BASE    = "/ShangGaoAIProjects/Lingge/LINCS/data"
PROC    = f"{BASE}/Processed_data"
OUT_DIR = f"{PROC}/L1000gctx_process"

CAT_PATH = f"{OUT_DIR}/Y_landmark_train_catalog_canonical.index.parquet"
CAT = pd.read_parquet(CAT_PATH)

# --- 剂量：原值 + log10(μM+ε) ---
eps = 1e-8
dose = CAT["dose_uM"].astype(float).to_numpy()
dose_log10 = np.log10(dose + eps)
X_dose = np.stack([dose, dose_log10], axis=1).astype(np.float32)
np.save(f"{OUT_DIR}/X_context_dose.npy", X_dose)
print("Saved:", f"{OUT_DIR}/X_context_dose.npy", X_dose.shape)

# --- 细胞：整数索引（训练时用 nn.Embedding） ---
cells = CAT["cell_id"].astype(str).fillna("NA").to_list()
uniq_cells = sorted(set(cells))
cell_to_idx = {c:i for i,c in enumerate(uniq_cells)}
X_cell_idx = np.array([cell_to_idx[c] for c in cells], dtype=np.int32)

np.save(f"{OUT_DIR}/X_context_cell_idx.npy", X_cell_idx)
with open(f"{OUT_DIR}/cell_to_idx.json", "w") as f:
    json.dump(cell_to_idx, f, indent=2)

print("Saved:", f"{OUT_DIR}/X_context_cell_idx.npy", X_cell_idx.shape)
print("Saved:", f"{OUT_DIR}/cell_to_idx.json", f"(n_cells={len(uniq_cells)})")


Saved: /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/X_context_dose.npy (209540, 2)
Saved: /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/X_context_cell_idx.npy (209540,)
Saved: /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/L1000gctx_process/cell_to_idx.json (n_cells=42)


In [9]:
import os, shutil, json
import numpy as np
import pandas as pd

BASE     = "/ShangGaoAIProjects/Lingge/LINCS/data"
PROC     = f"{BASE}/Processed_data"
SRC_DIR  = f"{PROC}/L1000gctx_process"
DEST_DIR = f"{PROC}/training_data"
os.makedirs(DEST_DIR, exist_ok=True)

# === 源文件清单（已存在于 SRC_DIR 的产出） ===
files = {
    # 目录（索引）
    "catalog_with_scaffolds": f"{SRC_DIR}/Y_landmark_train_catalog_canonical_with_scaffolds.index.parquet",
    # 标签
    "Y_mmap":                 f"{SRC_DIR}/Y_landmark_train_catalog_canonical.mmap",
    # 分子向量（raw 与 L2 任选其一或都带上）
    "X_mol_raw":              f"{SRC_DIR}/X_mol_molformer_768d.npy",
    "X_mol_l2":               f"{SRC_DIR}/X_mol_molformer_768d_l2.npy",
    # 剂量 & 细胞
    "X_dose":                 f"{SRC_DIR}/X_context_dose.npy",
    "X_cell_idx":             f"{SRC_DIR}/X_context_cell_idx.npy",
    "cell_to_idx":            f"{SRC_DIR}/cell_to_idx.json",
    # MoLFormer 唯一库（可选，便于溯源/增量编码）
    "mol_unique_emb":         f"{SRC_DIR}/molformer_unique_embeddings.npy",
    "uniq_smiles_map":        f"{SRC_DIR}/uniq_smiles_with_emb_row.parquet",
    # Landmark 与行重排资料（便于溯源/复算）
    "landmark_idx_p1":        f"{SRC_DIR}/landmark_idx_phase1.npy",
    "landmark_idx_p2":        f"{SRC_DIR}/landmark_idx_phase2_in_p2coords.npy",
    "landmark_ids":           f"{SRC_DIR}/landmark_gene_ids_phase1.txt",
}

# 校验存在性
missing = [k for k, p in files.items() if not os.path.exists(p)]
if missing:
    print("⚠️ 以下文件不存在，请先确认前面步骤已生成：", missing)

# 链接/复制到目标目录
def link_or_copy(src, dst):
    try:
        if os.path.islink(dst) or os.path.exists(dst):
            os.remove(dst)
        os.symlink(src, dst)   # 软链接（Linux）
    except Exception:
        shutil.copy2(src, dst)

for name, src in files.items():
    if not os.path.exists(src): 
        continue
    dst = os.path.join(DEST_DIR, os.path.basename(src))
    link_or_copy(src, dst)

print("Packed into:", DEST_DIR)
print("Files:", sorted(os.listdir(DEST_DIR)))


Packed into: /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/training_data
Files: ['X_context_cell_idx.npy', 'X_context_dose.npy', 'X_mol_molformer_768d.npy', 'X_mol_molformer_768d_l2.npy', 'Y_landmark_train_catalog_canonical.mmap', 'Y_landmark_train_catalog_canonical_with_scaffolds.index.parquet', 'cell_to_idx.json', 'landmark_gene_ids_phase1.txt', 'landmark_idx_phase1.npy', 'landmark_idx_phase2_in_p2coords.npy', 'molformer_unique_embeddings.npy', 'uniq_smiles_with_emb_row.parquet']


In [13]:
import os, json
import pandas as pd

BASE     = "/ShangGaoAIProjects/Lingge/LINCS/data"
PROC     = f"{BASE}/Processed_data"
SRC_DIR  = f"{PROC}/L1000gctx_process"
DEST_DIR = f"{PROC}/training_data"
os.makedirs(DEST_DIR, exist_ok=True)

# 优先使用带 scaffold 的 catalog；没有就退回原 catalog
cat_with_scaf = "Y_landmark_train_catalog_canonical_with_scaffolds.index.parquet"
catalog_name = cat_with_scaf if os.path.exists(f"{DEST_DIR}/{cat_with_scaf}") or os.path.exists(f"{SRC_DIR}/{cat_with_scaf}") \
               else "Y_landmark_train_catalog_canonical.index.parquet"

# 规范化：如果 catalog 不在 DEST_DIR，就从 SRC_DIR 软链/复制一份（若已有则跳过）
def ensure_in_dest(filename):
    import shutil
    src = os.path.join(SRC_DIR, filename)
    dst = os.path.join(DEST_DIR, filename)
    if not os.path.exists(dst):
        if os.path.exists(src):
            try:
                os.symlink(src, dst)
            except Exception:
                shutil.copy2(src, dst)
    return dst

catalog_path = os.path.join(DEST_DIR, catalog_name)
if not os.path.exists(catalog_path):
    catalog_path = ensure_in_dest(catalog_name)

CAT = pd.read_parquet(catalog_path)
N = len(CAT)
D_y = 978
D_mol = 768

# 构建 manifest：标注文件“格式”方便加载器分辨
manifest = {
    "version": "2025-09-30",
    "paths": {
        "catalog":       os.path.basename(catalog_path),
        "Y_mmap":        "Y_landmark_train_catalog_canonical.mmap",
        "X_mol_raw":     "X_mol_molformer_768d.npy",       # 注意：这是 memmap 写的“无头”二进制
        "X_mol_l2":      "X_mol_molformer_768d_l2.npy",    # 同上（如果存在）
        "X_dose":        "X_context_dose.npy",             # 标准 .npy
        "X_cell_idx":    "X_context_cell_idx.npy",         # 标准 .npy
        "cell_to_idx":   "cell_to_idx.json",
        "mol_unique_emb":"molformer_unique_embeddings.npy",   # 标准 .npy（唯一 SMILES 嵌入库）
        "uniq_smiles_map":"uniq_smiles_with_emb_row.parquet",
        "landmark_idx_p1":"landmark_idx_phase1.npy",
        "landmark_idx_p2":"landmark_idx_phase2_in_p2coords.npy",
        "landmark_ids":  "landmark_gene_ids_phase1.txt"
    },
    "formats": {  # 关键：声明读法
        "catalog":       "parquet",
        "Y_mmap":        "memmap",
        "X_mol_raw":     "memmap",
        "X_mol_l2":      "memmap",
        "X_dose":        "npy",
        "X_cell_idx":    "npy",
        "cell_to_idx":   "json",
        "mol_unique_emb":"npy",
        "uniq_smiles_map":"parquet",
        "landmark_idx_p1":"npy",
        "landmark_idx_p2":"npy",
        "landmark_ids":  "txt"
    },
    "shapes": {
        "N_rows": N,
        "Y":      [N, D_y],
        "X_mol":  [N, D_mol],
        "X_dose": [N, 2],
        "X_cell_idx": [N]
    },
    "feature_schema": {
        "catalog_columns": list(CAT.columns),
        "mol_embedding": {"dim": D_mol, "type": "float32", "normalized": False},
        "dose_features":  {"columns": ["dose_uM", "log10_dose_uM"], "dim": 2},
        "cell_index":     {"dtype": "int32", "embedding_dim_suggest": 64}
    }
}

with open(os.path.join(DEST_DIR, "manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)

print("Rewrote manifest with memmap formats ->", os.path.join(DEST_DIR, "manifest.json"))
print("N =", N, "| D_y =", D_y, "| D_mol =", D_mol)


Rewrote manifest with memmap formats -> /ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/training_data/manifest.json
N = 209540 | D_y = 978 | D_mol = 768


In [14]:
import os, json, numpy as np, pandas as pd

DEST_DIR = "/ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/training_data"

with open(os.path.join(DEST_DIR, "manifest.json")) as f:
    mani = json.load(f)

N = mani["shapes"]["N_rows"]
D_y = mani["shapes"]["Y"][1]
D_mol = mani["shapes"]["X_mol"][1]

def load_array(key):
    """根据 manifest.formats 选择加载方式"""
    path = os.path.join(DEST_DIR, mani["paths"][key])
    fmt  = mani["formats"][key]
    if fmt == "memmap":
        if key == "Y_mmap":
            return np.memmap(path, mode="r", dtype="float32", shape=(N, D_y))
        elif key in ("X_mol_raw","X_mol_l2"):
            return np.memmap(path, mode="r", dtype="float32", shape=(N, D_mol))
        else:
            raise ValueError(f"memmap shape for {key} not defined.")
    elif fmt == "npy":
        return np.load(path)
    elif fmt == "json":
        import json;  return json.load(open(path))
    elif fmt == "parquet":
        return pd.read_parquet(path)
    elif fmt == "txt":
        return open(path).read().splitlines()
    else:
        raise ValueError(f"Unknown format: {fmt}")

# 读取关键数据
CAT    = load_array("catalog")
Y      = load_array("Y_mmap")
X_mol  = load_array("X_mol_raw")
X_dose = load_array("X_dose")
X_cell = load_array("X_cell_idx")

print("Shapes OK?",
      Y.shape==(N, D_y),
      X_mol.shape==(N, D_mol),
      X_dose.shape==(N, 2),
      X_cell.shape==(N,))
print("Y mean/std:", float(Y.mean()), float(Y.std()))
print("X_mol mean/std:", float(X_mol.mean()), float(X_mol.std()))
print("Dose head:", X_dose[:3])
print("Cell idx head:", X_cell[:10])


Shapes OK? True True True True
Y mean/std: 0.0024143001064658165 1.3626172542572021
X_mol mean/std: -0.002106014173477888 0.5803754925727844
Dose head: [[ 0.37037    -0.43136418]
 [ 1.11111     0.04575706]
 [10.          1.        ]]
Cell idx head: [5 5 5 5 5 5 5 5 5 5]


In [1]:
import pandas as pd

In [4]:
OUT_DIR = "/ShangGaoAIProjects/Lingge/LINCS/data/Processed_data/training_data"
CAT = pd.read_parquet(f"{OUT_DIR}/Y_landmark_train_catalog_canonical_with_scaffolds.index.parquet")

In [8]:
CAT

,sig_id,phase,pert_id,pert_iname,cell_id,time_h,dose_uM,dose_value,dose_unit_raw,smiles,smiles_canonical,inchi_key,compound_id,is_small_molecule,is_control,pert_type,col_pos,dataset,scaffold
0,AML001_CD34_24H:BRD-A03772856:0.37037,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,0.37037,0.37037,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434316,GSE92742,O=C1C2CCCN2C(c2ccccc2)N1c1ccccc1
1,AML001_CD34_24H:BRD-A03772856:1.11111,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,1.11111,1.11111,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434315,GSE92742,O=C1C2CCCN2C(c2ccccc2)N1c1ccccc1
2,AML001_CD34_24H:BRD-A03772856:10,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,10.00000,10.00000,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434313,GSE92742,O=C1C2CCCN2C(c2ccccc2)N1c1ccccc1
3,AML001_CD34_24H:BRD-A03772856:3.33333,GSE92742,BRD-A03772856,BRD-A03772856,cd34,24.0,3.33333,3.33333,µM,COc1ccccc1C2N(C(=O)C3CCCN23)c4ccc(Cl)cc4,COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21,YSPMFQJSWDVJME-UHFFFAOYSA-N,YSPMFQJSWDVJME-UHFFFAOYSA-N,True,False,trt_cp,434314,GSE92742,O=C1C2CCCN2C(c2ccccc2)N1c1ccccc1
4,AML001_CD34_24H:BRD-A19037878:1.11111,GSE92742,BRD-A19037878,trichostatin-a,cd34,24.0,1.11111,1.11111,µM,CC(\C=C(C)\C=C\C(=O)NO)C(=O)c1ccc(cc1)N(C)C,CC(C=CC(=O)NO)=CC(C)C(=O)c1ccc(N(C)C)cc1,RTKIYFITIVXBLE-WKWSCTOISA-N,RTKIYFITIVXBLE-WKWSCTOISA-N,True,False,trt_cp,434463,GSE92742,c1ccccc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209535,REP.A028_YAPC_24H:K09,GSE70138,BRD-K60230970,MG-132,yapc,24.0,20.00000,20.00000,um,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,TZYWCYJVHRLUCT-VABKMULXSA-N,TZYWCYJVHRLUCT-VABKMULXSA-N,True,False,trt_cp,73903,GSE70138,c1ccccc1
209536,REP.A028_YAPC_24H:M18,GSE70138,BRD-K96862998,pirfenidone,yapc,24.0,0.04000,0.04000,um,Cc1ccc(=O)n(c1)-c1ccccc1,Cc1ccc(=O)n(-c2ccccc2)c1,ISWRGOKTTBVCFA-UHFFFAOYSA-N,ISWRGOKTTBVCFA-UHFFFAOYSA-N,True,False,trt_cp,73904,GSE70138,O=c1ccccn1-c1ccccc1
209537,REP.A028_YAPC_24H:O01,GSE70138,BRD-K60230970,MG-132,yapc,24.0,20.00000,20.00000,um,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,TZYWCYJVHRLUCT-VABKMULXSA-N,TZYWCYJVHRLUCT-VABKMULXSA-N,True,False,trt_cp,73905,GSE70138,c1ccccc1
209538,REP.A028_YAPC_24H:O06,GSE70138,BRD-K60230970,MG-132,yapc,24.0,20.00000,20.00000,um,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,TZYWCYJVHRLUCT-VABKMULXSA-N,TZYWCYJVHRLUCT-VABKMULXSA-N,True,False,trt_cp,73906,GSE70138,c1ccccc1


In [7]:
CAT.smiles.tolist()[209535]

'CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)OCc1ccccc1)C=O'